In [1]:
# from os import environ
# environ['OMP_NUM_THREADS'] = '12'
import cupy as cp
import numpy as np
import scipy.linalg as la

In [2]:
A = np.array([
    [1.5, 0.5],
    [0.5, 1.5],
    ]).astype('complex')
U = np.array(la.expm(1j*A))
eigs_u, vecs_u = np.linalg.eig(U)
U = cp.array(la.expm(1j*A))
print(vecs_u[1])

[-0.70710678+1.73472348e-16j  0.70710678+3.33066907e-16j]


In [3]:
#states
state_zero = cp.array([[1.0],[0.0]])
state_one = cp.array([[0.0],[1.0]])
#projectors
P0 = cp.dot(state_zero, state_zero.T)
P1 = cp.dot(state_one, state_one.T)

In [4]:
def multi_kron(*args):
    ret = cp.array([[1.0]]).astype('complex')
    for q in args:
        ret = cp.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = cp.eye(cp.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = cp.dot(ret, q)
    return ret

In [5]:
def H():
    return 1/cp.sqrt(2) * cp.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return cp.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return cp.array([[cp.exp(-1j*phi/2), 0],
                     [0, cp.exp(1j*phi/2)]]).astype('complex')
def P(phi):
    return cp.array([[1, 0],
                     [0, np.exp(1j*phi)]]).astype('complex')
def Ry(theta):
    return cp.array([[cp.cos(theta/2), -cp.sin(theta/2)],
                     [cp.sin(theta/2), cp.cos(theta/2)]]).astype('complex')
def I():
    return cp.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    for i in range(n_of_qubits):
        if i == control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i == target_q:
            list_of_ops_left.append(I())
            list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return (multi_kron(*list_of_ops_left) +
            multi_kron(*list_of_ops_right).astype('complex'))

In [6]:
phi = np.pi/3
cp.array([[1, 0],
          [0, np.exp(1j*phi)]]).astype('complex')

array([[1. +0.j       , 0. +0.j       ],
       [0. +0.j       , 0.5+0.8660254j]])

In [7]:
#eigenvalues to phases
def eigenvalues_to_phases(n, u):
    repetitions = 1
    ops = []
    for i in range(0, n):
        for _ in range(repetitions):
            ops.append(control(control_q=n-i,
                               target_q=0,
                               n_of_qubits=n+1,
                               operator=u**1))
        repetitions *= 2
    return multi_dot(*ops)

In [8]:
from qiskit import QuantumCircuit, Aer, execute
def swap(n_of_qubs):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    qc = QuantumCircuit(n_of_qubs+1)
    for qubit in range(n_of_qubs//2): #n//2
        qc.swap(qubit, n_of_qubs-qubit-1)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    op = result.get_unitary(qc).data
    return cp.array(op)

def iqft(n):
    ops = []
    hadamards = cp.array([I() for _ in range(n+1)])
    for j in range(n): #n
        for m in range(j):
            ops.append(
                control(
                control_q=n-m,
                target_q=n-j,
                n_of_qubits=n+1,
                operator=P(-cp.pi/float(2**(j-m)))
                )
            )
        hadamards_ = hadamards.copy()
        hadamards_[n-j] = H()
        ops.append(multi_kron(*hadamards_))
    return multi_dot(*ops)

In [ ]:
# import pydevd_pycharm
# pydevd_pycharm.settrace('localhost', port=1234, stdoutToServer=True, stderrToServer=True)
qubit_num = 9
prime_state = multi_kron(
    cp.array([[el] for el in vecs_u[1]]),
    state_zero, state_zero, state_zero,
    state_zero, state_zero, state_zero, state_zero, state_zero,
    state_zero)
#state preparation
state_prep = multi_kron(I(), *[H() for _ in range(qubit_num)] )
#calculations
eigs_to_phases = eigenvalues_to_phases(qubit_num, U)
iqft_matrix = iqft(qubit_num)
ift = multi_dot(swap(qubit_num), iqft_matrix)
final_u = multi_dot(ift, eigs_to_phases, state_prep)
final_state = cp.dot(final_u, prime_state)
rho = cp.dot(final_state, cp.conj(final_state).T)

In [ ]:
max_num = 2**qubit_num
max_prob = 0
eigenvalue = -9999
for i in range(max_num):
    ops = [I()]
    bin_int = "{0:b}".format(i).zfill(qubit_num)
    ops.extend([P0 if bit == '0' else P1 for bit in bin_int])
    proj = multi_kron(*ops)
    prob = np.trace(np.dot(proj, rho))
    if prob > max_prob:
        max_prob = prob
        eigenvalue = la.expm(1j*2*np.pi*i/(2**qubit_num))
print(f'calculated eigenvalue is {np.round(eigenvalue, 3)[0]}')
print(f'real eigenvalus are {np.round(eigs_u, 3)}')